In [3]:
!pip install transformers -q
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

2024-06-26 05:22:41.070161: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-26 05:22:41.070276: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-26 05:22:41.330849: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="main") # which version of model to use in repo

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [7]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [8]:
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [9]:
data = load_dataset("Amod/mental_health_counseling_conversations")

Generating train split:   0%|          | 0/3512 [00:00<?, ? examples/s]

In [14]:
data

DatasetDict({
    train: Dataset({
        features: ['Context', 'Response'],
        num_rows: 3512
    })
})

In [15]:
df = load_dataset("shawhin/shawgpt-youtube-comments")

Generating train split:   0%|          | 0/50 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9 [00:00<?, ? examples/s]

In [16]:
df

DatasetDict({
    train: Dataset({
        features: ['example'],
        num_rows: 50
    })
    test: Dataset({
        features: ['example'],
        num_rows: 9
    })
})

In [29]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

# craft prompt
comment = "i think i am not fit in this world"
prompt=f'''<s>[INST] {comment} [/INST]'''

# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s><s> [INST] i think i am not fit in this world [/INST] I'm here to listen and support you. It's important to remember that everyone has moments of feeling overwhelmed or uncertain about their place in the world. It's natural to have ups and downs in life. If you're feeling this way, it might be helpful to talk to someone about how you're feeling. This could be a trusted friend or family member, or a mental health professional. They can help you explore the reasons behind your feelings and provide you with strategies to cope. Remember, it's okay to ask for help when you need it. You are important and valued, and there are people who care about you. If you're having thoughts of


In [30]:
intstructions_string=f"""You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)



[INST] You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
 
i think i am not fit in this world 
[/INST]


In [31]:
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=140)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
 
i think i am not fit in this world 
[/INST] I'm really sorry to hear that you're feeling this way, but I want you to know that it's important to remember that every person has value and worth, regardless of how they may feel about themselves in the moment. It's natural to have ups and downs in life, and it's okay to not feel like you fit in perfectly all the time.

One thing that might help is to focus on the things that bring you joy and fulfillment. It could be a hobby, a relationship, or even just a simple a

In [102]:
train_dataset = data["train"]

# Convert the 'train' dataset to a list of dictionaries
train_list = train_dataset.to_dict()

context_list=train_list['Context']
response_list=train_list['Response']


In [103]:
instructions_string=f"""You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
"""
training_examples=[]
test_examples=[]
for context,response in zip(context_list,response_list):
    example=f"<s>[INST] {instructions_string} \n{context} \n[/INST] \n{response} \n</s>"
    if len(training_examples) <=200:
        training_examples.append(example)
    elif len(test_examples) <=25:
        test_examples.append(example) 
    else:
        break

In [ ]:
print(training_examples[0])

In [ ]:
print(test_examples[0])

In [106]:
tokenizer.pad_token=tokenizer.eos_token
tokenizer.truncation_side='left'
training_encodings=tokenizer(training_examples,return_tensors='np',truncation=True,padding='max_length',max_length=512)

In [107]:
test_encodings=tokenizer(training_examples,return_tensors='np',truncation=True,padding='max_length',max_length=512)

In [108]:
import torch
from torch.utils.data import Dataset,DataLoader

In [109]:
class textDataset(Dataset):
    def __init__(self,encodings):
        self.encodings=encodings
    def __len__(self):
        return len(self.encodings['input_ids'])
    
    def __getitem__(self,idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item      

In [ ]:
train_data=textDataset(training_encodings)
train_data[1]

In [111]:
test_data=textDataset(test_encodings)

In [112]:
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [113]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [114]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7929


In [115]:
lr = 2e-4
batch_size = 4
num_epochs = 10

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir= "/kaggle/working/",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",
    report_to=None
)

In [116]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_data,
    eval_dataset=test_data,
    args=training_args,
    data_collator=data_collator
)




In [117]:
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,3.113900,2.308430
1,1.919100,1.705455
2,1.608300,1.574059
4,1.530900,1.403755
5,1.362700,1.357315
6,1.311100,1.328080
8,1.386600,1.299816
9,1.180000,1.298420


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

In [118]:
model.eval()
trainer.evaluate()

{'eval_loss': 1.2984201908111572,
 'eval_runtime': 127.8842,
 'eval_samples_per_second': 1.572,
 'eval_steps_per_second': 0.399,
 'epoch': 9.411764705882353}

In [130]:
from huggingface_hub import notebook_login
notebook_login()

In [131]:
hf_name = 'mohityadav' # your hf username or org name
model_id = hf_name + "/" + "mental-health-advisor-gpt"

In [132]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

Upload 8 LFS files:   0%|          | 0/8 [00:00<?, ?it/s]

events.out.tfevents.1719385345.adc95cf38b20.34.0:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

events.out.tfevents.1719385423.adc95cf38b20.34.1:   0%|          | 0.00/5.48k [00:00<?, ?B/s]

...

events.out.tfevents.1719385648.adc95cf38b20.34.2:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

events.out.tfevents.1719386347.adc95cf38b20.34.4:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

events.out.tfevents.1719391099.adc95cf38b20.34.5:   0%|          | 0.00/354 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/mohityadav/working/commit/a1847921592fd44bf07dbb87771cc87dff72b97c', commit_message='mohityadav/mental-health-advisor-gpt', commit_description='', oid='a1847921592fd44bf07dbb87771cc87dff72b97c', pr_url=None, pr_revision=None, pr_num=None)

In [124]:
intstructions_string=f"""You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
"""
comment="I am not feeling good, i think life is boring"
prompt=f'''<s>[INST] {comment} [/INST]'''
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

prompt = prompt_template(comment)
print(prompt)


[INST] You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
 
I am not feeling good, i think life is boring 
[/INST]


In [125]:
model.eval()
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=200)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] You are a compassionate and knowledgeable mental health advisor. Someone is sharing their mental 
state or expressing doubts about their well-being. Provide the best advice and suggestions to help them navigate their
feelings. Offer empathetic, practical, and actionable guidance, and recommend helpful resources or techniques.
Ensure your response is simple, relatable, encourages openness, and provides reassurance.
please provide the valuable advice for this comment.
 
I am not feeling good, i think life is boring 
[/INST] 
Life is not supposed to be boring. It is a gift.  It is a journey.  It is a chance to learn and grow.  It is a chance to make a difference.  It is a chance to be happy.  It is a chance to be loved.  It is a chance to be successful.  It is a chance to be creative.  It is a chance to be a good person.  It is a chance to be a hero.  It is a chance to be a hero to someone.  It is a chance to be a hero to yourself.  It is a chance to be a hero to the world.  It